# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nuuk,64.1835,-51.7216,27.21,68,100,2.30,2.30,1701840057
1,1,mafinga,-7.2500,35.0667,71.40,81,100,1.48,1.48,1701840123
2,2,waitangi,-43.9535,-176.5597,56.82,80,100,10.00,10.00,1701840124
3,3,inari,68.9060,27.0288,5.20,98,98,1.77,1.77,1701840124
4,4,yellowknife,62.4560,-114.3525,24.42,95,100,4.61,4.61,1701840124


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

city_map = city_data_df.hvplot.points("Lng", "Lat", geo = True, size = "Humidity",scale = 1,color = "City",
    alpha = 0.7,
    frame_width = 850,
    frame_height = 500,
    tiles = "OSM",
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values
nice_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 32) & (city_data_df["Max Temp"] > 21)                
    & (city_data_df["Cloudiness"] < 50) 
    & (city_data_df["Humidity"] < 80)                               
                                  ]
# Drop any rows with null values
nice_weather_df = nice_weather_df.dropna()

# Display sample data
nice_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,floro,61.5996,5.0328,31.96,69,40,11.50,11.50,1701839706
54,54,changling,44.2653,124.0005,21.72,62,10,14.07,14.07,1701840145
89,89,longyearbyen,78.2186,15.6401,24.64,68,20,21.85,21.85,1701839572
95,95,kodiak,57.7900,-152.4072,29.71,55,0,40.29,40.29,1701839537
144,144,xinyuan,37.3000,99.0000,27.45,24,0,15.95,15.95,1701840176
181,181,yelizovo,53.1873,158.3805,24.94,39,0,13.42,13.42,1701839693
378,378,vilyuchinsk,52.9306,158.4028,26.87,64,8,9.53,9.53,1701840259
538,538,tromso,69.6496,18.9570,23.02,68,40,14.97,14.97,1701840316


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = nice_weather_df[["City", "Country", "Lat", "Lng", "Humidity", "Max Temp", "Cloudiness"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Max Temp,Cloudiness,Hotel Name
53,floro,11.50,61.5996,5.0328,69,31.96,40,
54,changling,14.07,44.2653,124.0005,62,21.72,10,
89,longyearbyen,21.85,78.2186,15.6401,68,24.64,20,
95,kodiak,40.29,57.7900,-152.4072,55,29.71,0,
144,xinyuan,15.95,37.3000,99.0000,24,27.45,0,
181,yelizovo,13.42,53.1873,158.3805,39,24.94,0,
378,vilyuchinsk,9.53,52.9306,158.4028,64,26.87,8,
538,tromso,14.97,69.6496,18.9570,68,23.02,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
floro - nearest hotel: Comfort Hotel Victoria Florø
changling - nearest hotel: No hotel found
longyearbyen - nearest hotel: The Vault
kodiak - nearest hotel: Shelikof Lodge
xinyuan - nearest hotel: No hotel found
yelizovo - nearest hotel: Николина Усадьба
vilyuchinsk - nearest hotel: Победа
tromso - nearest hotel: Clarion Collection Hotel Aurora


,City,Country,Lat,Lng,Humidity,Max Temp,Cloudiness,Hotel Name
53,floro,11.50,61.5996,5.0328,69,31.96,40,Comfort Hotel Victoria Florø
54,changling,14.07,44.2653,124.0005,62,21.72,10,No hotel found
89,longyearbyen,21.85,78.2186,15.6401,68,24.64,20,The Vault
95,kodiak,40.29,57.7900,-152.4072,55,29.71,0,Shelikof Lodge
144,xinyuan,15.95,37.3000,99.0000,24,27.45,0,No hotel found
181,yelizovo,13.42,53.1873,158.3805,39,24.94,0,Николина Усадьба
378,vilyuchinsk,9.53,52.9306,158.4028,64,26.87,8,Победа
538,tromso,14.97,69.6496,18.9570,68,23.02,40,Clarion Collection Hotel Aurora


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [55]:
%%capture --no-display

hover_city_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, size = "Humidity",scale = 1,color = "City",
    alpha = 0.8,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    hover_cols = ["Max Temp", "Hotel Name", "Country"]
)

# Display the map
hover_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp,Hotel Name,Country)